In [1]:
import xml.etree.ElementTree as ET
import yaml
import nmf_gym
from pathlib import Path

In [2]:
df3dpp_to_nmf_names = {
    'ThC_pitch': 'Coxa', 'ThC_roll': 'Coxa_roll', 'ThC_yaw': 'Coxa_yaw',
    'CTr_pitch': 'Femur', 'CTr_roll': 'Femur_roll',
    'FTi_pitch': 'Tibia', 'TiTa_pitch': 'Tarsus1'
}
nmf_to_df3dpp_names = {v: k for k, v in df3dpp_to_nmf_names.items()}

In [3]:
# Load SDF
basedir = Path(nmf_gym.__path__[0]).absolute().parent
sdf = ET.parse(basedir / 'data/design/sdf/neuromechfly_42dof.sdf')
root = sdf.getroot()

In [4]:
# Load joint limits
with open(basedir / 'data/pose/joint_limits.yaml') as f:
    joint_limits_dict = yaml.safe_load(f)

In [5]:
# Modify joint limits in SDF
model = root[0]
for child in model:
    if child.tag == 'joint' and child.attrib['type'] == 'revolute':
        joint_name = child.attrib['name'].replace('joint_', '')
        side = joint_name[0]
        pos = joint_name[1]
        nmf_dof_name = joint_name[2:]
        df3dpp_dof_name = nmf_to_df3dpp_names[nmf_dof_name]
        vmin = joint_limits_dict[pos][df3dpp_dof_name]['min']
        vmax = joint_limits_dict[pos][df3dpp_dof_name]['max']
        if side == 'R':
            if nmf_dof_name.endswith('_roll') or nmf_dof_name.endswith('_yaw'):
                vmin *= -1
                vmax *= -1
        if vmax < vmin:
            vmin, vmax = vmax, vmin
        print(f'{joint_name}: limit set to ({vmin}, {vmax})')
        limit = child.find('axis').find('limit')
        limit.find('lower').text = str(vmin)
        limit.find('upper').text = str(vmax)

LFCoxa: limit set to (-0.26425672291328106, 0.48552702413105564)
LFCoxa_roll: limit set to (0.16618682344461239, 1.3785967194737108)
LFCoxa_yaw: limit set to (-0.5804031973442535, 0.10931022152116496)
LFFemur: limit set to (-2.6674538515956217, -0.8634161259065807)
LFFemur_roll: limit set to (-0.09402975125252772, 0.8634003305866089)
LFTarsus1: limit set to (-0.4945172993154937, 0.03844435237973801)
LFTibia: limit set to (0.007870165035474774, 1.8518722717610507)
LHCoxa: limit set to (-0.15240583905787586, 0.33929149509972634)
LHCoxa_roll: limit set to (1.8681302587854358, 2.8534725490623054)
LHCoxa_yaw: limit set to (-0.4241460454516544, 0.1835604834168864)
LHFemur: limit set to (-2.973344491854622, -0.966381929159005)
LHFemur_roll: limit set to (-0.5672651974503836, 0.23888150024645177)
LHTarsus1: limit set to (-1.0252381241569277, 0.13229666920923713)
LHTibia: limit set to (0.06859821848423248, 3.0921175399378398)
LMCoxa: limit set to (-0.2747082182541584, 0.019250639876072107)
LMCo

In [6]:
# Save new SDF
sdf.write(basedir / 'data/design/sdf/neuromechfly_42dof_with_limit.sdf',
          xml_declaration=True)